In [ ]:
import os
import os.path as osp
import subprocess
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Rain Data Exploration

The purpose of this notebook is to explore the rain data from RAWS and HRRR retrieval processes.

* RAWS Rain Data:
    * [RAWS Definitions](https://www.weather.gov/media/wrh/mesowest/MesoWest_Data_Variables_Definitions.pdf)
    * They list hourly rainfall as parameters, but in practice stations only have wide data availability for accumulated precipitation
    * This data is potentially flawed when the rain collecting apparatus is full, and will thus register zero new rainfall when the bucket is full.
* HRRR Rain Data:
    * [HRRR Definitions](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml)
    * Both rate and total ($kg\; m^{-2}s^{-1}$ and $kg\; m^{-2}$, respectively)

## Read FMDA Data Dictionary

In [ ]:
if not osp.exists("co_202306.pkl"):
    print("Retrieving FMDA data")
    subprocess.call("wget -P . https://demo.openwfm.org/web/data/fmda/dicts/co_202306.pkl", shell=True)
    assert osp.exists("co_202306.pkl")    
    print("Downloaded https://demo.openwfm.org/web/data/fmda/dicts/co_202306.pkl as co_202306.pkl")

dat = pd.read_pickle("co_202306.pkl")

In [ ]:
dat.keys()

## Get Rain Data from a station

In [ ]:
x = dat["CPTC2_202306"]
print(x.keys())
print("Station location info:")
print(x["loc"])

In [ ]:
print(x["RAWS"].keys())
rain_raws = x["RAWS"]["rain"]
precipa_raws = x["RAWS"]["precip_accum"]

In [ ]:
print(x["HRRR"].keys())
rain_hrrr = x["HRRR"]["rain"]
precipa_hrrr = x["HRRR"]["precip_accum"]

### Check Time lines up

In [ ]:
np.mean(x["RAWS"]["time"] == x["HRRR"]["time"])

In [ ]:
# Print max time difference between queried time and actual time returned by RAWS
np.max(np.array([datetime.strptime(val, '%Y-%m-%dT%H:%M:%SZ') for val in x["RAWS"]["time_raws"]]) - np.array([datetime.strptime(val, '%Y-%m-%dT%H:%M:%SZ') for val in x["HRRR"]["time"]]))

## Plot together

In [ ]:
plt.plot(precipa_raws, label = "RAWS")
plt.plot(precipa_hrrr, label = "HRRR")
plt.legend()
plt.title("Hourly Rain Accumulated Data")